In [3]:
import requests
import pandas as pd
from datetime import datetime

# make a GET request to CoinGecko to get Bitcoin data
# extracts and returns the current circulating BTC supply
def get_btc_supply():
  """
    Fetch current circulating supply for Bitcoin from CoinGecko
  """
  url = "https://api.coingecko.com/api/v3/coins/bitcoin"
  resp = requests.get(url)  # Call CoinGecko API for Bitcoin info
  data = resp.json()
  return data["market_data"]["circulating_supply"] # Extract circulating supply of bitcoin

def fetch_market_data(days, interval=None):
  """
    Fetch Bitcoin price and volume data from CoinGecko market_chart endpoint
  """
  url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
  params = {
        "vs_currency": "usd",
        "days": days
    }
  if interval:
    params["interval"] = interval
  response = requests.get(url, params=params)

  if response.status_code != 200:
    raise Exception(f"API request failed: {response.status_code} - {response.text}")

  data = response.json()

  if "prices" not in data:
      raise Exception(f"'prices' key missing in response: {data}")

  return data


def compute_mvrv(data, btc_supply):
  """
    Compute MVRV ratio for Bitcoin.
  """
  # Extracting prices and volume from the JSON data
  prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
  volumes = pd.DataFrame(data["total_volumes"], columns=["timestamp", "volume"])

  # Add a new datetime column that converts the milliseconds timestamp into human readable format
  prices["datetime"] = pd.to_datetime(prices["timestamp"], unit="ms")
  volumes["datetime"] = pd.to_datetime(volumes["timestamp"], unit="ms")

  # Merging Price and Volume by Time
  df = pd.merge(prices[["datetime", "price"]], volumes[["datetime", "volume"]], on="datetime")

  # Calculate value = price * volume
  df["value_contrib"] = df["price"] * df["volume"]

  # Calculate cumulative value
  df["cum_value"] = df["value_contrib"].cumsum()

  # Calculate cumulative volume
  df["cum_volume"] = df["volume"].cumsum()

  # Calculates VWAP (Volume-Weighted Average Price) = cumulative value / cumulative volume.
  df["vwap"] = df["cum_value"] / df["cum_volume"]

  # Calculate Realized Value
  df["realized_value"] = df["vwap"] * btc_supply

  # Calculate Market Value
  df["market_value"] = df["price"] * btc_supply

  # Calculate MVRV = Market Value/Realized Value
  df["mvrv_ratio"] = df["market_value"] / df["realized_value"]

  return df[["datetime", "price", "volume", "vwap", "realized_value", "market_value", "mvrv_ratio"]]


def main():
    print("Fetching BTC circulating supply...")
    btc_supply = get_btc_supply()
    print(f"BTC Supply: {btc_supply:,.0f} coins")

    print("Fetching daily data (max)...")
    daily_data = fetch_market_data(days=365)
    daily_mvrv = compute_mvrv(daily_data, btc_supply)
    daily_mvrv.to_csv("daily_mvrv.csv", index=False)
    print("Saved daily MVRV to daily_mvrv.csv")

    print("Fetching hourly data (last 90 days)...")
    hourly_data = fetch_market_data(days=90)
    hourly_mvrv = compute_mvrv(hourly_data, btc_supply)
    hourly_mvrv.to_csv("hourly_mvrv.csv", index=False)
    print("Saved hourly MVRV to hourly_mvrv.csv")

if __name__ == "__main__":
    main()


Fetching BTC circulating supply...
BTC Supply: 19,884,196 coins
Fetching daily data (max)...
Saved daily MVRV to daily_mvrv.csv
Fetching hourly data (last 90 days)...
Saved hourly MVRV to hourly_mvrv.csv


In [4]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Load daily mvrv csv of Bitcoin
df = pd.read_csv("daily_mvrv.csv", parse_dates=["datetime"])

# Create subplot with 2 rows: Price/VWAP on top, MVRV ratio below
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=(" Daily Market Price & Realized Price (VWAP) over Time", "Daily MVRV Ratio over Time"))

fig.add_trace(go.Scatter(x=df["datetime"], y=df["price"], mode="lines", name=" Market Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["vwap"], mode="lines", name="Realized Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["mvrv_ratio"], mode="lines", name="MVRV Ratio"), row=2, col=1)

# Add a horizontal line at MVRV=1 to mark fair value
fig.add_shape(type="line", x0=df["datetime"].min(), x1=df["datetime"].max(),
              y0=1, y1=1, yref='y2', line=dict(color="Red", dash="dash"))

fig.update_layout(height=600, width=900, title_text="Daily Bitcoin MVRV Dashboard", template="plotly_white")
fig.show()


In [5]:
# Load hourly Bitcoin MVRV CSV
df = pd.read_csv("hourly_mvrv.csv", parse_dates=["datetime"])

# Create 2 plots Market Price vs Realized Price on top, MVRV ratio below
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Hourly Market Price & Realized Price over Time", "Hourly MVRV Ratio over Time")
)

fig.add_trace(go.Scatter(x=df["datetime"], y=df["price"], mode="lines", name="Market Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["vwap"], mode="lines", name="Realized Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["mvrv_ratio"], mode="lines", name="MVRV Ratio"), row=2, col=1)

# Horizontal line at MVRV = 1 (fair value)
fig.add_shape(type="line",
              x0=df["datetime"].min(), x1=df["datetime"].max(),
              y0=1, y1=1,
              yref='y2',
              line=dict(color="Red", dash="dash"),
              row=2, col=1)

fig.update_layout(height=600, width=900, title_text="Hourly Bitcoin MVRV Dashboard", template="plotly_white")
fig.update_yaxes(range=[0.5, df["mvrv_ratio"].max() * 1.1], row=2, col=1)

fig.show()
